# Basic Volcanic Tectonic Event Detection

## Steps

1. Create a catalog of events from location, start time, and end time.
2. Add a known event to the catalog
3. Create an inventory of stations from location, start time, and end time.
4. Loop over events and create picks for each channel based on distance of the station from source and a very simple assumption of 5 km/s wave speed.
5. Make the templates with eqcorrscan. Construct templates with a 10-second window to capture P and S with a bit of filtering.
6. Run the templates through continuous data and see what wasn't located by the network.

## Setup

We'll use two Python packages. Obspy supports accessing seismic data and performing common operations such as picking. The second package is eqcorrscan, a package for creating templates for finding similar events in waveform data. Obspy is installed in GeoLab, but eqcorrscan must be installed using the conda package manager.

In [ ]:
!conda install -c conda-forge eqcorrscan

Import the packages.

In [1]:
import numpy as np
import eqcorrscan

from eqcorrscan import Tribe
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
from obspy.core.event import (
    Catalog, Event, Pick, WaveformStreamID, Origin)
from obspy.geodetics import gps2dist_azimuth

## Finding Events

Obspy can find events with a location, expressed as coordinates, a start time, and an end time. For this example, we will examine volcanic tectonic events at Mt St. Helens using iMUSH (Imaging Magma Under Mt. St. Helens)data. IMush collected seismic data caused by earthquakes and active source from 2014 to 2016, providing a rich data source for volcanic tectonic activity. We will start with located events from July 17-19 2014 under Mount St. Helens (17 located events).

In [2]:
client = Client("USGS")
catalog = client.get_events(
    starttime=UTCDateTime(2014, 7, 17),
    endtime=UTCDateTime(2014, 7, 19),
    latitude=46.2, longitude=-122.19,
    maxradius=0.25)

print(f"Total events found: {catalog.count()}")

Total events found: 17


> Explanation
>
> The code uses the obspy `Client` function to connect and query the USGS FDSN service. The `get_events` method sends a query to the USGS service with the location of Mt. St. Helens, the start time, the end time, and the distance from the location in kilometers. The query returns a list or catalog of 17 events.

Events at volcanoes are difficult to locate, especially if they're small. If you don't have a template, you'll never find them with template detection. We will use a known event triggered at 2014-07-25T16:57:57.08 found using RedPy, a Python software for automated detection and analysis of repeating earthquakes in continuous data. The known event acts a seed for creating templates. We don't have a location, so let's just assume it's near the summit at a depth of about 2 km b.s.l.

The following code creates the event and adds it to the catalog.

In [3]:
event_solo = Event()
event_solo.origins = [Origin()]
event_solo.origins[0].time = UTCDateTime('2014-07-25T16:57:57.08') - 2  # assume about 2 seconds from origin to trigger time
event_solo.origins[0].latitude = 46.2
event_solo.origins[0].longitude = -122.19
event_solo.origins[0].depth = 2000

# Append to the catalog
catalog += event_solo

# print(catalog)
print(catalog.__str__(print_all=True))

18 Event(s) in Catalog:
2014-07-18T22:48:15.530000Z | +46.201, -122.181 | -0.47 mh | manual
2014-07-18T21:34:55.770000Z | +46.199, -122.183 | 0.35 ml | manual
2014-07-18T06:08:15.750000Z | +46.200, -122.198 | -0.31 mh | manual
2014-07-18T06:07:15.470000Z | +46.202, -122.201 | -0.22 mh | manual
2014-07-18T03:26:32.930000Z | +46.142, -122.163 | 0.0  md | manual
2014-07-18T00:31:47.200000Z | +46.200, -122.176 | -0.3  mh | manual
2014-07-17T21:50:20.370000Z | +46.190, -122.123 | -0.14 mh | manual
2014-07-17T20:24:49.000000Z | +46.200, -122.179 | 0.29 ml | manual
2014-07-17T20:22:03.670000Z | +46.197, -122.172 | -0.36 mh | manual
2014-07-17T19:53:37.290000Z | +46.194, -122.186 | -0.12 mh | manual
2014-07-17T19:30:26.160000Z | +46.200, -122.188 | -0.28 mh | manual
2014-07-17T19:26:16.270000Z | +46.197, -122.177 | -0.14 mh | manual
2014-07-17T19:12:12.210000Z | +46.303, -122.227 | -0.5  md | manual
2014-07-17T17:42:17.610000Z | +46.200, -122.181 | 0.26 ml | manual
2014-07-17T15:07:17.380000Z 

> Explanation:
>
> The code creates an obspy Event and adds it to the catalog. The known event is a seed for creating a collection of templates, called a Tribe 

## Make Templates

Pick information for these earthquakes is not available. To create templates we'll have to make synthetic picks using a very rough estimate the P-wave arrival time at the locations of our stations of interest. The picks are the starting point of a template window. This is a good method for finding more events, bit if you were doing a big relative relocation study you'd work with actual pick times and shorter windows around P- and S-.

### Create an Inventory of Stations

We will need an inventory of stations within .25 km around Mt. St. Helens. Because volcanic-tectonic earthquakes are often shallow, weak motion, high-frequency events, we're interested in high gain, vertical component of a waveform.

> Explanation:
>
> Obspy queries the Earthscope FDSN dataselect server for stations within .25 kilometers of Mt. St. Helens between July 17th and July 19th, 2014. The request limits it to stations with high frequency channels with a vertical component. The number of stations found are shown, along with the total number of channels found. Finally, the contents of a channel are shown, which is useful for explaining code in a following example.

### Download waveforms using the inventory

In [6]:
import os

from matplotlib import pyplot as plt
from obspy import UTCDateTime, read_inventory
from obspy.clients.fdsn import mass_downloader
from obspy.clients.fdsn.mass_downloader import (
    CircularDomain,
    MassDownloader,
    RectangularDomain,
    Restrictions,
)

domain = CircularDomain(
    latitude=46.2,
    longitude=-122.19,
    minradius=0.0,
    maxradius=0.25  # degrees (~55 km at this latitude)
)

# Get unique networks and stations
# networks = set()
# stations = set()

# for network in inventory:
#     networks.add(network.code)
#     for station in network:
#         stations.add(station.code)

# network_list = ",".join(sorted(networks))
# station_list = ",".join(sorted(stations))

# print(network_list)

network_list = ["XD"]
station_list = ["M*"]

restrictions = mass_downloader.Restrictions(
    starttime=UTCDateTime("2014-07-17"),
    endtime=UTCDateTime("2014-07-26"),
    chunklength_in_sec=86400.0,
    network="XD,CC,PB,UW",
    location="*",
    channel="*HZ",
    station="M*",
    reject_channels_with_gaps=False,
    minimum_length=0.0,
    minimum_interstation_distance_in_m=100.0,
    sanitize=True
)

# Downloader instance
downloader = mass_downloader.MassDownloader(providers=["IRIS"])

# Download
downloader.download(
    domain,
    restrictions,
    mseed_storage="./seismic_data/waveform_data",
    stationxml_storage="./seismic_data/station_metadata"
)



[2026-01-28 22:46:35,898] - obspy.clients.fdsn.mass_downloader - INFO: Initializing FDSN client(s) for IRIS.
[2026-01-28 22:46:35,898] - obspy.clients.fdsn.mass_downloader - INFO: Initializing FDSN client(s) for IRIS.
[2026-01-28 22:46:35,898] - obspy.clients.fdsn.mass_downloader - INFO: Initializing FDSN client(s) for IRIS.
[2026-01-28 22:46:35,909] - obspy.clients.fdsn.mass_downloader - INFO: Successfully initialized 1 client(s): IRIS.
[2026-01-28 22:46:35,909] - obspy.clients.fdsn.mass_downloader - INFO: Successfully initialized 1 client(s): IRIS.
[2026-01-28 22:46:35,909] - obspy.clients.fdsn.mass_downloader - INFO: Successfully initialized 1 client(s): IRIS.
[2026-01-28 22:46:35,911] - obspy.clients.fdsn.mass_downloader - INFO: Total acquired or preexisting stations: 0
[2026-01-28 22:46:35,911] - obspy.clients.fdsn.mass_downloader - INFO: Total acquired or preexisting stations: 0
[2026-01-28 22:46:35,911] - obspy.clients.fdsn.mass_downloader - INFO: Total acquired or preexisting s

{'IRIS': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0x135bcfb10>}

### Create inventory

In [7]:
from obspy import read_inventory
from obspy.core.inventory import Inventory
from pathlib import Path

metadata_dir = Path('./seismic_data/station_metadata')

filepaths = list(metadata_dir.rglob('*.xml'))
filepaths = [str(fp) for fp in filepaths] 

inventory = Inventory()

for filename in filepaths:
    try:
        # Read the inventory from the file
        station = read_inventory(filename)
        # Add the networks from the current inventory to the merged inventory
        inventory.networks.extend(station.networks)
        # inventory = inventory.merge(station)
    except Exception as e:
        print(f"Error reading {filename}: {e}")

# (Optional) You can use the merge method to handle overlapping stations/epochs, 
# although the simple extend often works if they are distinct files.
# merged_inventory = merged_inventory.merge(inv_to_merge_with) 

# Write the combined inventory to a new StationXML file
output_filename = "./seismic_data/station_metadata/stations.xml"
inventory.write(output_filename, format="StationXML")

print(f"Stations found: {len(inventory)}")
channel_count = 0
for network in inventory:
    for station in network:
        for channel in station:
            channel_count += 1

print(f"Total Channels Found: {channel_count}")

for channel in inventory.get_contents()['channels']:
    print(channel)

Stations found: 27
Total Channels Found: 53
UW.MTM..EHZ
XD.MF05..HHZ
XD.MF05..LHZ
XD.MG03..HHZ
XD.MG03..LHZ
XD.MG05..HHZ
XD.MG05..LHZ
XD.MG06..HHZ
XD.MG06..LHZ
XD.MG07..HHZ
XD.MG07..LHZ
XD.MG08..HHZ
XD.MG08..LHZ
XD.MH04..HHZ
XD.MH04..LHZ
XD.MH07..HHZ
XD.MH07..LHZ
XD.MI04..HHZ
XD.MI04..LHZ
XD.MI05..HHZ
XD.MI05..LHZ
XD.MI07..HHZ
XD.MI07..LHZ
XD.MI08..HHZ
XD.MI08..LHZ
XD.MI09..HHZ
XD.MI09..LHZ
XD.MJ06..HHZ
XD.MJ06..LHZ
XD.MJ07..HHZ
XD.MJ07..LHZ
XD.MJ09..HHZ
XD.MJ09..LHZ
XD.MK04..HHZ
XD.MK04..LHZ
XD.MK06..HHZ
XD.MK06..LHZ
XD.MK07..HHZ
XD.MK07..LHZ
XD.MK08..HHZ
XD.MK08..LHZ
XD.ML07..HHZ
XD.ML07..LHZ
XD.ML09..HHZ
XD.ML09..LHZ
XD.MM04..HHZ
XD.MM04..LHZ
XD.MM06..HHZ
XD.MM06..LHZ
XD.MM08..HHZ
XD.MM08..LHZ
XD.MN07..HHZ
XD.MN07..LHZ


### Create picks

In [8]:
"""
Add synthetic picks by reading local miniseed files
"""

from obspy import read, UTCDateTime
from obspy.core.event import Pick, WaveformStreamID
from obspy.geodetics import gps2dist_azimuth
from obspy import read_inventory
import numpy as np
import os
import glob

# Load inventory from a local file
inventory = read_inventory("./seismic_data/station_metadata/stations.xml")

p_vel = 5000  # m/s 
for event in catalog:
    picks = []
    for channel in inventory.get_contents()['channels']:
        # XD (iMUSH deployment) broadbands have LHZ, BHZ, and HHZ channels; we only need HHZ
        if channel[:2] != 'XD' or (channel[:2] == 'XD' and channel[-3:] == 'HHZ'):
            distance = np.sqrt(gps2dist_azimuth(
                event.origins[0].latitude,
                event.origins[0].longitude,
                inventory.get_channel_metadata(channel)['latitude'],
                inventory.get_channel_metadata(channel)['longitude']
            )[0]**2 + (event.origins[0].depth + inventory.get_channel_metadata(channel)['elevation'])**2)
            picks += [
                Pick(
                    time=event.origins[0].time +  distance/p_vel,
                    waveform_id=WaveformStreamID(
                        network_code=channel.split('.')[0],
                        station_code=channel.split('.')[1],
                        channel_code=channel.split('.')[3],
                        location_code=channel.split('.')[2],
                    ),
                    phase_hint='P'
                )
            ]
    event.picks = picks
print(len(event.picks))

# catalog.write("output_catalog_explicit.xml", format="QUAKEML")

# catalog.write("./seismic_data/basic_event_catalog.xml", format="QUAKEML")

27


### Create templates

In [9]:
# Make the templates with eqcorrscan

# We're constructing templates with a 10-second window to capture P and S with a bit of filtering.
# These can all be adjusted but should match the filtering and processing length you plan to use to
# scan through the continuous data. 

# There are a lot of stations and quite a few possible channels (40+)... we can reduce the search space if needed

from obspy import read
from pathlib import Path

catalog.write("./seismic_data/basic_event_catalog.xml", format="QuakeML")
data_dir = Path('./seismic_data/waveform_data/')

# Get all mseed files recursively
filepaths = list(data_dir.rglob('*.mseed'))

# Convert to strings if needed
filepaths = [str(fp) for fp in filepaths]    

for mseed in filepaths:
    st = read(mseed)
    
    tribe = Tribe().construct(
        method="from_meta_file",
        # method="from_client",
        # client_id="IRIS",
        # catalog=catalog,
        st=st,
        meta_file="./seismic_data/basic_event_catalog.xml",
        lowcut=1.0,
        highcut=10.0,
        samp_rate=50.0,
        filt_order=4,
        length=10.0,
        prepick=1,
        process_len=86400,
        all_horiz=False,
        parallel=True,
        min_snr=3.0,
        skip_short_chans=True
    )


print(len(tribe))

    
    

/Users/sophiaparafina/miniconda3/envs/ssa_prototype/lib/python3.13/site-packages/obspy/io/quakeml/core.py:1112: UserWarning: 'quakeml:earthquake.usgs.gov/fdsnws/event/1/query?starttime=2014-07-17T00%3A00%3A00.000000&endtime=2014-07-19T00%3A00%3A00.000000&latitude=46.2&longitude=-122.19&maxradius=0.25' is not a valid QuakeML URI. It will be in the final file but note that the file will not be a valid QuakeML file.
  warnings.warn(msg % obj.id)
Pick outside of data span: Pick time 2014-07-18T22:48:19.512929Z Start time 2014-07-25T00:00:00.045000Z End time: 2014-07-26T00:00:00.025000Z
Pick outside of data span: Pick time 2014-07-18T22:48:20.825565Z Start time 2014-07-25T00:00:00.045000Z End time: 2014-07-26T00:00:00.025000Z
Pick outside of data span: Pick time 2014-07-18T22:48:21.028882Z Start time 2014-07-25T00:00:00.045000Z End time: 2014-07-26T00:00:00.025000Z
Pick outside of data span: Pick time 2014-07-18T22:48:18.993529Z Start time 2014-07-25T00:00:00.045000Z End time: 2014-07-26T00

In [10]:
print(len(tribe))

0


## Finding New Events

Now we run the templates through continuous data and see what wasn't located by the network. There's a *lot* going on under the hood here...

In [ ]:
party = tribe.client_detect(
    client=client,
    starttime=UTCDateTime(2014,7,15),
    endtime=UTCDateTime(2014,8,10),
    threshold=10,
    threshold_type="MAD",
    trig_int=1.0,
    concurrent_processing=True,
)

In [ ]:
party.plot(plot_grouped=True)
party.plot(plot_grouped=False)

print(f'Number of templates: {len(tribe)}')
print(f'Number of additional detections: {len(party)-len(tribe)}')